In [11]:
import sys
import os
sys.path.append(os.path.abspath('/Users/ericxia/school/Math-148-Project/food-classification'))

import pandas as pd
from data_utils.utils import keep_existing_photos, downsample_group
from data_utils.dataset import PhotoLabelDataset, stratified_split_dataset, train_transform, val_transform
from model.resnet18 import Resnet18FineTuneModel
from model.utils import get_device, train_model_single_epoch, validate_model_single_epoch, save_checkpoint, evaluate_on_test

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

### Data preprocessing

In [12]:
base_dir = "../../"

business_df = pd.read_json(f'{base_dir}data/yelp_dataset/yelp_academic_dataset_business.json', lines=True)
photos_df = pd.read_json(f'{base_dir}data/yelp_photos/photos.json', lines=True)

photo_dir = f"{base_dir}data/yelp_photos/resized_photos"
photos_df = keep_existing_photos(photos_df, photo_dir)

photos_df = photos_df[photos_df['label'] == 'food'].copy()

categories_df = business_df[['business_id', 'attributes']].copy()
photos_df = photos_df.merge(categories_df, on="business_id", how="left")

photos_df = photos_df[photos_df['attributes'].notna()]
photos_df['price_range'] = photos_df['attributes'].apply(lambda x: x.get('RestaurantsPriceRange2'))

photos_df['price_range'] = photos_df['price_range'].astype(int)
photos_df['price_range'] = photos_df['price_range'].replace({2: 1, 3: 2, 4: 2}) # Binary classification
photos_df.price_range.value_counts()

Checking images: 100%|██████████| 200100/200100 [00:05<00:00, 39271.43it/s]


price_range
1    88822
2     8107
Name: count, dtype: int64

In [13]:
# Downsample to get balanced dataset
price_2_num = (photos_df.price_range == 2).sum()

food_df = photos_df.groupby('price_range', group_keys=False).apply(
    lambda x: downsample_group(x, price_2_num) if x.name in [1] else x  # Only downsample for price_range 1 and 2
)

food_df['price_range'] = food_df['price_range'] - 1

/var/folders/6n/sm2tfzkn2hjd8cpfn_256ph00000gp/T/ipykernel_73184/1009235296.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  food_df = photos_df.groupby('price_range', group_keys=False).apply(


In [14]:
food_df.price_range.value_counts()

price_range
0    8107
1    8107
Name: count, dtype: int64

In [15]:
label = 'price_range'

price_dataset = PhotoLabelDataset(food_df, photo_dir, label)
labels = food_df[label].values 
train_size = 0.85
val_size = 0.10

train_dataset, val_dataset, test_dataset = stratified_split_dataset(
    price_dataset,
    labels,
    train_size=train_size,
    val_size=val_size,
    random_state=42
)

train_dataset.transform = train_transform
val_dataset.transform = val_transform
test_dataset.transform = val_transform

batch_size = 64

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
)

for images, labels in train_loader:
    print("Images shape:", images.shape)  
    print("Labels shape:", labels.shape)
    break

Images shape: torch.Size([64, 3, 224, 224])
Labels shape: torch.Size([64])


### Model training

In [16]:
num_classes = 2
device = get_device()

model = Resnet18FineTuneModel(num_classes=num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)
scaler = torch.amp.GradScaler("cuda")
grad_clip = 1
ckpt_dir = "checkpoints/price_2_classes"

history = {
    "train_loss": [],
    "train_accuracy": [],
    "val_loss": [],
    "val_accuracy": []
}

# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, device, scaler, grad_clip, history, ckpt_dir, num_epochs=10):
    model.train()

    for epoch in range(1, num_epochs+1):
        train_loss, train_accuracy = train_model_single_epoch(
            model, train_loader, criterion, optimizer, device, scaler, grad_clip
        )
        val_loss, val_accuracy = validate_model_single_epoch(model, val_loader, criterion, device)

        print(
            f"[Epoch {epoch}/{num_epochs}]",
            f"Train Loss: {train_loss:.4f} | " f"Train Accuracy: {train_accuracy:.4f}",
            f"Val Loss: {val_loss:.4f} | " f"Val Accuracy: {val_accuracy:.4f}"
        )

        history["train_loss"].append(train_loss)
        history["train_accuracy"].append(train_accuracy)
        history["val_loss"].append(val_loss)
        history["val_accuracy"].append(val_accuracy)

        save_checkpoint(epoch, model, optimizer, history, ckpt_dir)

/Users/ericxia/Library/Caches/pypoetry/virtualenvs/food-classification-nES7ttkF-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ericxia/Library/Caches/pypoetry/virtualenvs/food-classification-nES7ttkF-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/ericxia/Library/Caches/pypoetry/virtualenvs/food-classification-nES7ttkF-py3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  

In [19]:
num_epochs = 20
train_model(model, train_loader, val_loader, criterion, optimizer, device, scaler, grad_clip, history, ckpt_dir, num_epochs=num_epochs)

Training: 100%|██████████| 216/216 [00:29<00:00,  7.30it/s, loss=0.486]


[Epoch 1/20] Train Loss: 0.5948 | Train Accuracy: 0.7087 Val Loss: 0.5706 | Val Accuracy: 0.7250
Model checkpoint saved at checkpoints/price_2_classes/ckpt_1


Training: 100%|██████████| 216/216 [00:29<00:00,  7.20it/s, loss=0.362]


[Epoch 2/20] Train Loss: 0.4792 | Train Accuracy: 0.7754 Val Loss: 0.6589 | Val Accuracy: 0.7158
Model checkpoint saved at checkpoints/price_2_classes/ckpt_2


Training: 100%|██████████| 216/216 [00:30<00:00,  7.15it/s, loss=0.239]


...

[Epoch 18/20] Train Loss: 0.0310 | Train Accuracy: 0.9893 Val Loss: 1.4966 | Val Accuracy: 0.7454
Model checkpoint saved at checkpoints/price_2_classes/ckpt_18


Training: 100%|██████████| 216/216 [00:30<00:00,  7.06it/s, loss=0.0322]  


[Epoch 19/20] Train Loss: 0.0280 | Train Accuracy: 0.9886 Val Loss: 1.5386 | Val Accuracy: 0.7472
Model checkpoint saved at checkpoints/price_2_classes/ckpt_19


Training: 100%|██████████| 216/216 [00:30<00:00,  7.01it/s, loss=0.331]   


[Epoch 20/20] Train Loss: 0.0285 | Train Accuracy: 0.9901 Val Loss: 1.6913 | Val Accuracy: 0.7398
Model checkpoint saved at checkpoints/price_2_classes/ckpt_20


### Model evaluation

In [23]:
ckpt = torch.load(f"{base_dir}saved_models/price_binary/ckpt_best")
model = Resnet18FineTuneModel(num_classes=num_classes)
model.load_state_dict(ckpt['model_state_dict'])
model.to(device)
model.eval()

/Users/ericxia/Library/Caches/pypoetry/virtualenvs/food-classification-nES7ttkF-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ericxia/Library/Caches/pypoetry/virtualenvs/food-classification-nES7ttkF-py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Resnet18FineTuneModel(
  (base): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True

In [22]:
evaluate_on_test(model, test_loader, device, ['0', '1'])

Classification Report on Test Set:
              precision    recall  f1-score   support

           0       0.77      0.66      0.71       406
           1       0.70      0.80      0.75       405

    accuracy                           0.73       811
   macro avg       0.74      0.73      0.73       811
weighted avg       0.74      0.73      0.73       811

